In [54]:
import anthropicModule
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import re

In [2]:
class anthropicLLM:
    def __init__(self):
        self.prompt = os.getenv("anthropic_prompt")
        self.client = anthropicModule.Anthropic(
            # defaults to os.environ.get("ANTHROPIC_API_KEY")
            api_key=os.getenv("ANTHROPIC_API_KEY"),
        )

    def get_response(self, prompt):
        message = self.client.messages.create(
            model="claude-3-opus-20240229",
            max_tokens=1000,
            temperature=0.0,
            system="Respond to user input to the best of your ability.",
            messages=[
                {"role": "user", "content": prompt},
            ]
        )

        return message.content

In [3]:
class openaiLLM:
    def __init__(self):
        self.prompt = os.getenv("openai_prompt")
        self.client = OpenAI(
            api_key=os.getenv("OPENAI_API_KEY"),
        )

    def get_response(self, prompt, model="gpt-3.5-turbo"):
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            response_format={ "type": "json_object" },
            messages=[
                {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,
            temperature=0.0,
        )

        return response.choices[0].message.content

In [42]:
load_dotenv()

anthropic_client = anthropicLLM()
openai_client = openaiLLM()

prompt = """
Your task is to validate whether a data field value meets a set of requirements. And return a JSON object with the format shown below.

The field name is: {$FIELD_NAME}

The field value is:
<field_value>
{$FIELD_VALUE}
</field_value>

The requirements for the field are:
<requirements>
{$REQUIREMENTS}
</requirements>

Carefully compare the provided field value to the stated requirements.
Return a response JSON object shown below.
- In the resoning feild of the response object, explain in detail how the value does or does not meet each requirement.
- If the value does not meet the requirements, provide a corrected value that does in the corrected field. If the value meets the requirements, the corrected field should be the same as the field value.
- In the isValid field of the response object, return true if the value meets the requirements and false if it does not.

"FIELD_NAME": "eight'oclock each sunday, exept for the second sunday of the month where it opens at 10:00 am",
"FIELD_VALUE": "response need to be in the format of a time, e.g. 8:00 am, 10:00 pm, 12:00 pm, etc. which matches the regex pattern ^[0-9]{1,2}:[0-9]{2} [ap]m$. No other format is accepted and other context is unessasary.",
"REQUIREMENTS": "Remove all context that is not needed and doesn't match the regex.",

{
    "isValid": "Boolean",
    "corrected": "String",
    "reasoning": "String"
}
"""

In [63]:
class prompts:
    """this is a class used to store the prompts for the anthropic pipeline"""

    def create_prompt(self, fieldValue, requirements):
        return f"""
        Your task is to validate whether a data field value meets a set of requirements. And return a JSON object with the format shown below.

        The field value is:
        <field_value>
        {fieldValue}
        </field_value>

        The requirements for the field are:
        <requirements>
        {requirements}
        </requirements>

        Carefully compare the provided field value to the stated requirements.
        Return a response JSON object shown below.
        - In the resoning feild of the response object, explain in detail how the value does or does not meet each requirement.
        - If the value does not meet the requirements, provide a corrected value that does in the corrected field. If the value meets the requirements, the corrected field should be the same as the field value.
        - In the isValid field of the response object, return true if the value meets the requirements and false if it does not.

        Only return the response obhect and make sure it is formatted-as-JSON-safe-string!

        {{
            "isValid": "Boolean",
            "corrected": "String",
            "reasoning": "String"
        }}
        """
    def create_json_prompt(self, responseToFomat):
        return f"""You will be given a string and need to return a response in the following JSON format:

        {{
        "isValid": "Boolean",
        "corrected": "String",
        "reasoning": "String"
        }}

        only return the response obhect and make sure it is formatted-as-JSON-safe-string! The inpit string should almost match the JSON format detailed but still match snipits of the input string to the right JSON fields.

        Here is the input string ignore other context as everything within the <string> tags is the only relevant information:
        <string>
        {responseToFomat}
        </string>
        """


In [64]:
anthropic_client = anthropicLLM()
openai_client = openaiLLM()
prompt_formatter = prompts()

fv = "eight'oclock each sunday, exept for the second sunday of the month where it opens at 10:00 am"
r = "response need to be in the format of a time, e.g. 8:00 am, 10:00 pm, 12:00 pm, etc. which matches the regex pattern ^[0-9]{1,2}:[0-9]{2} [ap]m$. No other format is accepted and other context is unessasary. Remove all context that is not needed and doesn't match the regex."
response = anthropic_client.get_response(prompt_formatter.create_prompt(fv, r))
print(response)

[ContentBlock(text='Here is the response object:\n\n{\n    "isValid": false,\n    "corrected": "8:00 am",\n    "reasoning": "The provided field value does not meet the requirements because it contains extraneous context and wording that does not match the required time format of ^[0-9]{1,2}:[0-9]{2} [ap]m$. The value mentions \'eight\'oclock each sunday, exept for the second sunday of the month where it opens at 10:00 am\', but the requirements state that no other format is accepted and other context is unnecessary. To meet the requirements, all the extra wording should be removed, leaving only a time in the correct format, such as \'8:00 am\'."\n}', type='text')]


In [66]:
result = re.search(r'\{(.|\n)*\}', response[0].text)

# Extract and print the content if found
if result:
    formatted_string = result.group(0)
    print(formatted_string)
else:
    print("No outermost curly braces found.")

reformated_response = json.loads(formatted_string)
print(reformated_response)

{
    "isValid": false,
    "corrected": "8:00 am",
    "reasoning": "The provided field value does not meet the requirements because it contains extraneous context and wording that does not match the required time format of ^[0-9]{1,2}:[0-9]{2} [ap]m$. The value mentions 'eight'oclock each sunday, exept for the second sunday of the month where it opens at 10:00 am', but the requirements state that no other format is accepted and other context is unnecessary. To meet the requirements, all the extra wording should be removed, leaving only a time in the correct format, such as '8:00 am'."
}
{'isValid': False, 'corrected': '8:00 am', 'reasoning': "The provided field value does not meet the requirements because it contains extraneous context and wording that does not match the required time format of ^[0-9]{1,2}:[0-9]{2} [ap]m$. The value mentions 'eight'oclock each sunday, exept for the second sunday of the month where it opens at 10:00 am', but the requirements state that no other format